In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.metrics as sklm
import xgboost as xgb
import matplotlib
import pickle

In [2]:
train = pd.read_csv('Train_maskedv2.csv')
test = pd.read_csv('Test_maskedv2.csv')
submissions = pd.read_csv('samplesubmissionv2.csv')
vd = pd.read_csv('variable_descriptions_v2.csv')

In [3]:
print(train.shape)
print(test.shape)

(3174, 50)
(1102, 49)


In [4]:
#train1 = pd.read_csv('Train_maskedv2.csv')
Labels = np.array(train['target_pct_vunerable'])
print(Labels)

[1.73980135 1.99338128 0.03834043 ... 0.95882823 1.02493356 5.28312496]


In [5]:
train = train.drop(['ward'], axis = 1)
test = test.drop(['ward'], axis = 1)

In [6]:
#excluded_cols = ['dw_03', 'dw_05', 'pg_02', 'lan_14', 'psa_03', 'lan_02', 'dw_02', 'dw_10', 
               # 'psa_02', 'lan_07', 'lan_09', 'lan_11', 'dw_11', 'lan_10']
    
excluded_cols = ['dw_12', 'dw_13', 'lan_13', 'dw_10', 'psa_02', 'lan_07', 'lan_09']
for i in excluded_cols:
    test = test.drop(i, axis=1)
    
for j in excluded_cols:
    train = train.drop(j, axis=1)

In [7]:
train = train.drop(['target_pct_vunerable'], axis=1)
print(train.shape)
print(test.shape)

(3174, 41)
(1102, 41)


In [8]:
train['avg_individuals'] = train['total_individuals'] / train['total_households']
test['avg_individuals'] = test['total_individuals'] / test['total_households']

In [9]:
Features = np.array(train)
test_enc = np.array(test)

In [10]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(Features[:, :2])
Features[:, :2] = scaler.transform(Features[:, :2])
print(Features[:, :2])

[[ 3.25349424  2.85925741]
 [ 3.2611637   2.1992301 ]
 [-0.23453747 -0.44538003]
 ...
 [ 0.93342628  1.8316802 ]
 [ 3.1076322   3.35482504]
 [ 0.25944328  0.27111737]]


In [11]:
scaler1 = preprocessing.StandardScaler().fit(test_enc[:, :2])
test_enc[:, :2] = scaler1.transform(test_enc[:, :2])
print(test_enc[:, :2])

[[-0.16503446 -0.35938299]
 [-0.42163757 -0.42024847]
 [-0.46203328 -0.6051864 ]
 ...
 [ 0.32568337  0.19777147]
 [-0.32950555 -0.10753076]
 [-0.38886785 -0.5674457 ]]


In [12]:
import numpy.random as nr
import sklearn.model_selection as ms
## Randomly sample cases to create independent training and test data
nr.seed(9988)
indx = range(Features.shape[0])
indx = ms.train_test_split(indx, test_size = 0.2)
x_train = Features[indx[0],:]
y_train = np.ravel(Labels[indx[0]])
x_test = Features[indx[1],:]
y_test = np.ravel(Labels[indx[1]])

In [13]:
indx1 = range(x_train.shape[0])
indx1 = ms.train_test_split(indx1, test_size = 0.2)
x_train1 = x_train[indx1[0],:]
y_train1 = np.ravel(y_train[indx1[0]])
x_train_val = x_train[indx1[1],:]
y_train_val = np.ravel(y_train[indx1[1]])

In [14]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=400, max_depth=10, max_features='auto',
                            min_samples_leaf=1, min_samples_split=2, random_state=0)
rf.fit(x_train, y_train)

C:\Users\hp\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [15]:
rf_pred = rf.predict(x_test)
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
mean_squared_error(rf_pred, y_test) ** 0.5

6.743862986511016

In [17]:
preds1 = rf.predict(test_enc)
submissions.target_pct_vunerable = preds1
submissions.to_csv('SouthAfricaCovid6.csv', index=False)

In [20]:
xg_reg = xgb.XGBRegressor(objective = 'reg:tweedie', n_estimators = 2000, seed = 123, max_depth = 10,
                           learning_rate=0.1, booster = 'gbtree', base_score = 0.7, subsample = 0.8,
                           reg_lambda = 0.03)

eval_set = [(x_test, y_test)]
xg_reg.fit(x_train, y_train, eval_metric="rmse", eval_set=eval_set, verbose=True, early_stopping_rounds = 200)

[0]	validation_0-rmse:15.92877
Will train until validation_0-rmse hasn't improved in 200 rounds.
[1]	validation_0-rmse:15.83912
[2]	validation_0-rmse:15.73394
[3]	validation_0-rmse:15.61222
[4]	validation_0-rmse:15.47247
[5]	validation_0-rmse:15.30789
[6]	validation_0-rmse:15.12141
[7]	validation_0-rmse:14.91141
[8]	validation_0-rmse:14.69171
[9]	validation_0-rmse:14.43861
[10]	validation_0-rmse:14.16035
[11]	validation_0-rmse:13.85646
[12]	validation_0-rmse:13.53306
[13]	validation_0-rmse:13.19042
[14]	validation_0-rmse:12.85338
[15]	validation_0-rmse:12.48311
[16]	validation_0-rmse:12.12287
[17]	validation_0-rmse:11.75373
[18]	validation_0-rmse:11.38580
[19]	validation_0-rmse:11.02601
[20]	validation_0-rmse:10.69190
[21]	validation_0-rmse:10.35728
[22]	validation_0-rmse:10.06251
[23]	validation_0-rmse:9.78651
[24]	validation_0-rmse:9.52631
[25]	validation_0-rmse:9.27993
[26]	validation_0-rmse:9.06464
[27]	validation_0-rmse:8.85297
[28]	validation_0-rmse:8.68208
[29]	validation_0-rmse

[257]	validation_0-rmse:7.47559
[258]	validation_0-rmse:7.47599
[259]	validation_0-rmse:7.47602
[260]	validation_0-rmse:7.47604
[261]	validation_0-rmse:7.47605
[262]	validation_0-rmse:7.47552
[263]	validation_0-rmse:7.47476
[264]	validation_0-rmse:7.47472
[265]	validation_0-rmse:7.47463
[266]	validation_0-rmse:7.47465
[267]	validation_0-rmse:7.47475
[268]	validation_0-rmse:7.47461
[269]	validation_0-rmse:7.47467
Stopping. Best iteration:
[69]	validation_0-rmse:7.40806



XGBRegressor(base_score=0.7, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
       importance_type='gain', interaction_constraints=None,
       learning_rate=0.1, max_delta_step=0, max_depth=10,
       min_child_weight=1, missing=nan, monotone_constraints=None,
       n_estimators=2000, n_jobs=0, num_parallel_tree=1,
       objective='reg:tweedie', random_state=123, reg_alpha=0,
       reg_lambda=0.03, scale_pos_weight=None, seed=123, subsample=0.8,
       tree_method=None, validate_parameters=False, verbosity=None)

In [21]:
rf_pred = x.predict(x_test)
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
mean_squared_error(rf_pred, y_test) ** 0.5

6.743862986511016